In [50]:
import pymysql
import pandas as pd
import numpy as np
import random

### 데이터베이스(MySQL) 연결

In [51]:
try:
    # MySQL 데이터베이스에 연결
    conn = pymysql.connect(
        host='capstone-db.cfgmik22w69x.ap-northeast-2.rds.amazonaws.com',
        user='root',
        password='daejin2019',
        db='capstone',
        charset='utf8',
        port=3306
    )
    
    if conn.open:  # is_connected() 대신 open 속성을 사용
        print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")

except pymysql.MySQLError as e:
    print(f"연결 중 오류 발생: {e}")

# finally:
#     if 'connection' in locals() and connection.open:
#         connection.close()
#         print("MySQL 데이터베이스와의 연결이 종료되었습니다.")


MySQL 데이터베이스에 성공적으로 연결되었습니다.


### Accommodation 테이블 생성, 테이터 삽입

In [3]:
cur = conn.cursor()

cur.execute("CREATE TABLE Accommodation (contentid INT PRIMARY KEY,title VARCHAR(255) NOT NULL, part VARCHAR(25) NOT NULL,area VARCHAR(255) NOT NULL,sigungu VARCHAR(255) NOT NULL,addr VARCHAR(255) NOT NULL,tel VARCHAR(20),firstimage VARCHAR(255),firstimage2 VARCHAR(255), minfee int(10))CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")

0

In [4]:
# CSV 파일 경로
csv_file_path = '../data/hotel_after_updated.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)
data.head(5)

,contentid,title,class,area,sigungu,addr,tel,firstimage,firstimage2,minfee
0,2465071,가경재,한옥,경상북도,안동시,경상북도 안동시 하회남촌길 69-5,054-855-8552,http://tong.visitkorea.or.kr/cms/resource/00/2...,http://tong.visitkorea.or.kr/cms/resource/00/2...,80000.0
1,142785,가락관광호텔,관광호텔,서울특별시,송파구,서울특별시 송파구 송파대로28길 5,02-400-6641~3,NaN,NaN,98000.0
2,2671267,가락청,한옥,전북특별자치도,전주시,전북특별자치도 전주시 완산구 한지길 68,NaN,NaN,NaN,50000.0
3,2627867,가람나무,한옥,경기도,파주시,경기도 파주시 소라지로327번길 126-21,NaN,NaN,NaN,87000.0
4,1865597,가람초연재,한옥,경상북도,안동시,경상북도 안동시 풍천면 하회종가길 76-6,010-3849-7542,http://tong.visitkorea.or.kr/cms/resource/48/2...,http://tong.visitkorea.or.kr/cms/resource/48/2...,100000.0


In [5]:
for index, row in data.iterrows():
    sql = """
        INSERT INTO Accommodation (contentid, title, part, area, sigungu, addr, tel, firstimage, firstimage2, minfee)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # NaN 값을 빈 문자열로 대체
    values = [value if pd.notna(value) else '' for value in row.values]
    print(values)  # 디버깅용 출력
    cur.execute(sql, values)

[2465071, '가경재 ', '한옥', '경상북도', '안동시', '경상북도 안동시 하회남촌길 69-5', '054-855-8552', 'http://tong.visitkorea.or.kr/cms/resource/00/2626200_image2_1.jpg', 'http://tong.visitkorea.or.kr/cms/resource/00/2626200_image3_1.jpg', 80000.0]
[142785, '가락관광호텔', '관광호텔', '서울특별시', '송파구', '서울특별시 송파구 송파대로28길 5', '02-400-6641~3', '', '', 98000.0]
[2671267, '가락청', '한옥', '전북특별자치도', '전주시', '전북특별자치도 전주시 완산구 한지길 68', '', '', '', 50000.0]
[2627867, '가람나무', '한옥', '경기도', '파주시', '경기도 파주시 소라지로327번길 126-21', '', '', '', 87000.0]
[1865597, '가람초연재', '한옥', '경상북도', '안동시', '경상북도 안동시 풍천면 하회종가길 76-6', '010-3849-7542', 'http://tong.visitkorea.or.kr/cms/resource/48/2993048_image2_1.jpg', 'http://tong.visitkorea.or.kr/cms/resource/48/2993048_image3_1.jpg', 100000.0]
[1896032, '가름게스트하우스', '펜션', '제주특별자치도', '서귀포시', '제주특별자치도 서귀포시 법환하로9번길 10', '064-739-4499', '', '', 55000.0]
[2571998, '가리벨리 관광펜션', '펜션', '강원특별자치도', '인제군', '강원특별자치도 인제군 인제읍 대목리길 137', '033-463-1212', 'http://tong.visitkorea.or.kr/cms/resource/37/2571137_image2_1.jpeg', 

In [6]:
cur.execute("SELECT COUNT(*) FROM Accommodation;")
a = cur.fetchall()
a

((3825,),)

In [7]:
conn.commit()

### Room 테이블 생성, 데이터 삽입

cur.execute("DROP TABLE Room;")
conn.commit()

In [57]:
cur = conn.cursor()

cur.execute("CREATE TABLE Room (contentid INT NOT NULL,roomcode INT NOT NULL,roomtitle VARCHAR(255) NOT NULL,roomcount INT,available_count INT NOT NULL,base_person INT NOT NULL,max_person INT NOT NULL,offseasonminnfee INT,offseasonmaxnfee INT,peakseasonminfee INT,peakseasonmaxfee INT,roombathfacility VARCHAR(1),roombath VARCHAR(1),roomhometheater VARCHAR(1),roomaircondition VARCHAR(1),roomtv VARCHAR(1),roompc VARCHAR(1),roomcable VARCHAR(1),roominternet VARCHAR(1),roomrefrigerator VARCHAR(1),roomtoiletries VARCHAR(1),roomsofa VARCHAR(1),roomcook VARCHAR(1),roomtable VARCHAR(1),roomhairdryer VARCHAR(1),rooming1 VARCHAR(255),rooming1_alt VARCHAR(255),rooming2 VARCHAR(255),rooming2_alt VARCHAR(255),rooming3 VARCHAR(255),rooming3_alt VARCHAR(255),PRIMARY KEY (contentid, roomcode));")
conn.commit()

In [58]:
# CSV 파일 경로
csv_file_path = '../data/room.csv'

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

In [59]:
triggers = [
    """
    DROP TRIGGER IF EXISTS `update_minfee_after_delete`$$
    CREATE TRIGGER `update_minfee_after_delete` 
    AFTER DELETE ON `Room` 
    FOR EACH ROW 
    BEGIN
        UPDATE Accommodation
        SET minfee = (
            SELECT MIN(offseasonminnfee)
            FROM Room
            WHERE contentid = OLD.contentid
        )
        WHERE contentid = OLD.contentid;
    END;
    $$""",
    
    """
    DROP TRIGGER IF EXISTS `update_minfee_after_insert`$$
    CREATE TRIGGER `update_minfee_after_insert` 
    AFTER INSERT ON `Room` 
    FOR EACH ROW 
    BEGIN
        UPDATE Accommodation
        SET minfee = (
            SELECT MIN(offseasonminnfee)
            FROM Room
            WHERE contentid = NEW.contentid
        )
        WHERE contentid = NEW.contentid;
    END;
    $$""",
    
    """
    DROP TRIGGER IF EXISTS `update_minfee_after_update`$$
    CREATE TRIGGER `update_minfee_after_update` 
    AFTER UPDATE ON `Room` 
    FOR EACH ROW 
    BEGIN
        UPDATE Accommodation
        SET minfee = (
            SELECT MIN(offseasonminnfee)
            FROM Room
            WHERE contentid = NEW.contentid
        )
        WHERE contentid = NEW.contentid;
    END;
    $$"""
]


In [61]:
for trigger in triggers:
    cur.execute(trigger)

conn.commit()
print("트리거가 성공적으로 생성되었습니다.")

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '$$\n    CREATE TRIGGER `update_minfee_after_delete` \n    AFTER DELETE ON `Room` \n' at line 1")

In [62]:
for index, row in data.iterrows():
    sql = """
        INSERT INTO Room (contentid, roomcode, roomtitle, roomcount, available_count, base_person, max_person, offseasonminnfee, offseasonmaxnfee, peakseasonminfee, peakseasonmaxfee, roombathfacility,roombath, roomhometheater, roomaircondition, roomtv, roompc, roomcable,roominternet,roomrefrigerator,roomtoiletries,roomsofa, roomcook, roomtable,roomhairdryer, rooming1,rooming1_alt, rooming2, rooming2_alt, rooming3, rooming3_alt)        
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # NaN 값을 빈 문자열로 대체
    values = [value if pd.notna(value) else '' for value in row.values]
    print(values)  # 디버깅용 출력
    cur.execute(sql, values)
conn.commit()

[2465071, 46591, '사랑채 아랫방', 6, 5, 2, 2, 80000, 122000, 127000, 174000, '', '', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', 'Y', 'Y', 'http://tong.visitkorea.or.kr/cms/resource/16/2626216_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)03', 'http://tong.visitkorea.or.kr/cms/resource/15/2626215_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)02', 'http://tong.visitkorea.or.kr/cms/resource/14/2626214_image2_1.jpg', '경북_안동_가경재_사랑채(아랫방)01']
[2465071, 46593, '군불 황토 아랫방', 6, 5, 3, 7, 100000, 141000, 108000, 154000, '', '', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', 'Y', 'Y', 'http://tong.visitkorea.or.kr/cms/resource/23/2626223_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)05', 'http://tong.visitkorea.or.kr/cms/resource/22/2626222_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)04', 'http://tong.visitkorea.or.kr/cms/resource/19/2626219_image2_1.jpg', '경북_안동_가경재_군불황토(아랫방)01']
[142785, 11432, '스탠다드 더블', 8, 2, 2, 3, 98000, 120000, 109000, 127000, 'Y', 'Y', '', 'Y', 'Y', '', 'Y', 'Y', 'Y', 'Y', '', '', '', 'Y', '', '', '', '', '', '']
[142785

In [63]:
cur.execute("SELECT COUNT(*) FROM Room;")
a = cur.fetchall()
a

((11752,),)

In [64]:
conn.commit()
conn.close()
print("MySQL 데이터베이스와의 연결이 종료되었습니다.")

MySQL 데이터베이스와의 연결이 종료되었습니다.
